# データ確認


- データ種類について
    - 合成人口データ（何かしらの処理済み)：2015_001_8_13208_01.csv
        - exe_pop_path 　/Users/osamu/study/poi_sim/data/exe_data

    - 合成人口データ　original 
        - pop_path 　/Users/osamu/study/poi_sim/data/origin/population
        - 10種類合成人口データ : 2015_001_8_13208.csv ~ 2015_010_8_13208.csv
            - 全て調布市のみ
        
    - 軌跡データ original(Home 付与済み)
        - すでにPoIのデータが付与されている状態
        - path  /Users/osamu/study/poi_sim/data/origin/trace
        - [Google Drive 阪上](https://drive.google.com/drive/folders/1Aow9ytnxKh0ZQOcnGGkJB693Mhp-o7-o)

    - 軌跡データ 加工済み
        - path /Users/osamu/study/poi_sim/data/exe_data/
        - ファイル命名形式：{yyyymm}_week{num}.csv　　(例) 201902_week1.csv 
            - home, outpit,pre,pre1, が存在　違いは不明
        - 2月〜4月の週ごとのデータを集約したもの
        
    - 遷移確率行列データ
        - path /Users/osamu/study/poi_sim/data/exe_data/new
        - ファイル命名形式：{num1}_{num2}.npy　　(例) 06_01.npy
            - num1: 06 ~ 11, num2: 01 ~ 04
            - 740×740 (PoI数×メッシュ数)
    - agoopの真値データ
        - path /Users/osamu/study/poi_sim/data/exe_data/true_data
        - ファイル命名形式：{num1}_fin.json　　(例) 06_fin.json
            - num1: 06 ~ 11

In [8]:
import pandas as pd
import os
import json
import csv
import glob
import collections
import matplotlib.pyplot as plt
import numpy as np

In [9]:
print(os.getcwd())  # 例: '/Users/username/projects'

/Users/osamu/study/poi_sim/src


### 0. データの読み込み

In [15]:
# Load

#軌跡データ　ファイル一覧取得
trace_path = "/Users/osamu/study/poi_sim/data/exe_data/"
files = sorted(os.listdir(trace_path))  
display(files)

# 各　軌跡データを読み込み　→ 調布市以外のデータもあり
# for file in files:
#     if file.endswith(".csv"):
#         print(file)
#         path = os.path.join(trace_path, file)  # フルパスを作る
#         df = pd.read_csv(path, index_col=0)
#         display(df.head(5))


# week データ内容確認　（defalt, home , output)
default_path = "/Users/osamu/study/poi_sim/data/exe_data/201902_week1.csv"
home_path    = "/Users/osamu/study/poi_sim/data/exe_data/201902_week1_home.csv"
output_path  = "/Users/osamu/study/poi_sim/data/exe_data/201902_week1_output.csv"

default_df = pd.read_csv(default_path, index_col=0)
home_df    = pd.read_csv(home_path, index_col=0)
output_df  = pd.read_csv(output_path, index_col=0)

display(default_df)
display(home_df)
display(output_df)


# orijginal の軌跡データの読み込み　→ PoIデータは付与済み
# trace_path = "/Users/osamu/study/poi_sim/data/origin/trace/20190201.csv"
# df = pd.read_csv(trace_path, index_col=0)
# display(df["workplace_citycode"].unique())
# display(df.head(5))

# data = np.load("/Users/osamu/study/poi_sim/data/exe_data/new/06_01.npy")
# print(type(data))   # データ型を確認
# print(data.shape)   # 配列の形を確認



['2015_001_8_13208_01.csv',
 '201902_week1.csv',
 '201902_week1_home.csv',
 '201902_week1_output.csv',
 '201902_week2.csv',
 '201902_week2_home.csv',
 '201902_week2_output.csv',
 '201902_week3.csv',
 '201902_week3_home.csv',
 '201902_week3_output.csv',
 '201902_week4.csv',
 '201902_week4_output.csv',
 '201902_week4_pre.csv',
 '201902_week4_pre1.csv',
 '201903_week1_pre_output.csv',
 '201903_week2_pre_output.csv',
 '201903_week3_pre_output.csv',
 '201903_week4_pre_output.csv',
 '201904_week1_output.csv',
 '201904_week2_output.csv',
 '201904_week3_output.csv',
 '201904_week4_output.csv',
 '201904_week5_output.csv',
 'new',
 'trans_prob_mat_element.json',
 'true_data']

,citycode,dailyid,home_citycode,judge,lat,lon,workplace_citycode,hour,min,meshid,大分類,中分類,小分類
0,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.660080,140.305565,12240.0,0,0,53403294,OoR,OoR,OoR
15,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,15,53404203,OoR,OoR,OoR
30,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,30,53404203,OoR,OoR,OoR
45,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.670679,140.302506,12240.0,0,45,53404204,NaN,NaN,NaN
60,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.648879,140.363540,12240.0,1,0,53403279,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9861968,13208.0,9a64b9c2232b7220dac59f7ac51abb78fa303edf7f6295...,13210.0,stay,35.660430,139.588558,13200.0,23,30,53393497,学校・趣味教室・図書館,塾・予備校,学習塾
9861983,13208.0,9a64b9c2232b7220dac59f7ac51abb78fa303edf7f6295...,13210.0,stay,35.660423,139.588553,13200.0,23,45,53393497,学校・趣味教室・図書館,塾・予備校,学習塾
9862002,13208.0,a3adfffeb39b5003862de1b3ec5fedf0fdb8c12a13b848...,13210.0,stay,35.656662,139.549884,14136.0,23,30,53393483,学校・趣味教室・図書館,塾・予備校,学習塾
9862017,13208.0,a3adfffeb39b5003862de1b3ec5fedf0fdb8c12a13b848...,13210.0,stay,35.656617,139.549972,14136.0,23,45,53393483,学校・趣味教室・図書館,塾・予備校,学習塾


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,citycode,dailyid,home_citycode,judge,lat,lon,workplace_citycode,hour,min,meshid,大分類,中分類,小分類,home_flag
0,0,0,0,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.660080,140.305565,12240.0,0,0,53403294,OoR,OoR,OoR,NaN
1,1,1,15,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,15,53404203,OoR,OoR,OoR,NaN
2,2,2,30,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,30,53404203,OoR,OoR,OoR,NaN
3,3,3,45,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.670679,140.302506,12240.0,0,45,53404204,Move,Move,Move,NaN
4,4,4,60,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.648879,140.363540,12240.0,1,0,53403279,Move,Move,Move,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2582670,2646769,3465205,7029565,13101.0,b8defa875615e0bea62439f4335f8499551471c4d17c93...,13224.0,stay,35.692383,139.766185,13100.0,17,30,53394631,医療・薬・保険衛生,医療機関,歯科,NaN
2582671,2646770,3465206,7029580,13101.0,b8defa875615e0bea62439f4335f8499551471c4d17c93...,13224.0,stay,35.692383,139.766185,13100.0,17,45,53394631,医療・薬・保険衛生,医療機関,歯科,NaN
2582672,2646771,3465207,7029595,13101.0,b8defa875615e0bea62439f4335f8499551471c4d17c93...,13224.0,stay,35.692383,139.766185,13100.0,18,0,53394631,医療・薬・保険衛生,医療機関,歯科,NaN
2582673,2646772,3465208,7029610,13101.0,b8defa875615e0bea62439f4335f8499551471c4d17c93...,13224.0,stay,35.692383,139.766185,13100.0,18,15,53394631,医療・薬・保険衛生,医療機関,歯科,NaN


,citycode,dailyid,home_citycode,judge,lat,lon,workplace_citycode,hour,min,meshid,大分類,中分類,小分類,home_flag
956824,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697556,139.519329,13210.0,0,15,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956839,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697585,139.519292,13210.0,0,30,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956854,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697555,139.519330,13210.0,0,45,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956869,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697555,139.519330,13210.0,1,0,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956884,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697554,139.519330,13210.0,1,15,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034796,13204.0,83dd2b90744461865c04a00c6802936425d7f0ad0dcc06...,13210.0,stay,35.662748,139.577779,13110.0,22,45,53393496,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
4034811,13204.0,83dd2b90744461865c04a00c6802936425d7f0ad0dcc06...,13210.0,stay,35.662748,139.577779,13110.0,23,0,53393496,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
4034826,13204.0,83dd2b90744461865c04a00c6802936425d7f0ad0dcc06...,13210.0,stay,35.662748,139.577779,13110.0,23,15,53393496,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
4034841,13204.0,83dd2b90744461865c04a00c6802936425d7f0ad0dcc06...,13210.0,stay,35.662748,139.577779,13110.0,23,30,53393496,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True


In [18]:
# display(default_df.count())
# display(home_df.count())
# display(output_df.count())

# display(len(default_df))
# display(len(home_df))
# display(len(output_df))


display(default_df.head(10))
display(home_df.head(10))
display(output_df.head(10))

,citycode,dailyid,home_citycode,judge,lat,lon,workplace_citycode,hour,min,meshid,大分類,中分類,小分類
0,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.660080,140.305565,12240.0,0,0,53403294,OoR,OoR,OoR
15,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,15,53404203,OoR,OoR,OoR
30,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,30,53404203,OoR,OoR,OoR
45,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.670679,140.302506,12240.0,0,45,53404204,NaN,NaN,NaN
60,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.648879,140.363540,12240.0,1,0,53403279,NaN,NaN,NaN
75,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,1,15,53403269,NaN,NaN,NaN
90,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,1,30,53403269,NaN,NaN,NaN
105,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,1,45,53403269,NaN,NaN,NaN
120,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,2,0,53403269,NaN,NaN,NaN
135,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,2,15,53403269,NaN,NaN,NaN


,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,citycode,dailyid,home_citycode,judge,lat,lon,workplace_citycode,hour,min,meshid,大分類,中分類,小分類,home_flag
0,0,0,0,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.660080,140.305565,12240.0,0,0,53403294,OoR,OoR,OoR,NaN
1,1,1,15,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,15,53404203,OoR,OoR,OoR,NaN
2,2,2,30,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,stay,35.666729,140.296133,12240.0,0,30,53404203,OoR,OoR,OoR,NaN
3,3,3,45,12230.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.670679,140.302506,12240.0,0,45,53404204,Move,Move,Move,NaN
4,4,4,60,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.648879,140.363540,12240.0,1,0,53403279,Move,Move,Move,NaN
5,5,5,75,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,1,15,53403269,Move,Move,Move,True
6,6,6,90,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,1,30,53403269,Move,Move,Move,True
7,7,7,105,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,1,45,53403269,Move,Move,Move,True
8,8,8,120,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,2,0,53403269,Move,Move,Move,True
9,9,9,135,12237.0,ad19eb00f178f6f164187f272557f4c3beb79ab2bd6fc8...,12240.0,move,35.641349,140.370581,12240.0,2,15,53403269,Move,Move,Move,True


,citycode,dailyid,home_citycode,judge,lat,lon,workplace_citycode,hour,min,meshid,大分類,中分類,小分類,home_flag
956824,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697556,139.519329,13210.0,0,15,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956839,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697585,139.519292,13210.0,0,30,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956854,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697555,139.519330,13210.0,0,45,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956869,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697555,139.519330,13210.0,1,0,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956884,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697554,139.519330,13210.0,1,15,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956899,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697554,139.519328,13210.0,1,30,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956914,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697553,139.519331,13210.0,1,45,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956929,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697553,139.519331,13210.0,2,0,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956944,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697553,139.519331,13210.0,2,15,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
956959,13210.0,2ef1b74ae1021591a70ab6ce506faffef5b7b2efd4a4e6...,13210.0,stay,35.697553,139.519331,13210.0,2,30,53394431,自動車・オートバイ・自転車・ドライブ,パーキング・駐輪場,駐車場業,True
